In [53]:
import IPython.core.display as di

# Hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# Add a button to toggle visibility of code blocks
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

# Extend width of page
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


Toggle code

In [56]:
import pandas as pd
import altair as alt

df = pd.read_csv('output/pm2_5_hourly_summary.csv')
df.head()

,Unnamed: 0,column_label,hour,mean_pm2_5,median_pm2_5
0,1,ry/Aoy 2 (outside) (31.75028 -106.48169) Primary,0,6.622125,4.320
1,2,ry/Aoy 2 (outside) (31.75028 -106.48169) Primary,1,8.406225,5.250
2,3,ry/Aoy 2 (outside) (31.75028 -106.48169) Primary,2,11.609209,6.070
3,4,ry/Aoy 2 (outside) (31.75028 -106.48169) Primary,3,11.968862,6.820
4,5,ry/Aoy 2 (outside) (31.75028 -106.48169) Primary,4,11.257910,6.345


In [39]:
# Filter to Zavala, Douglass, Mesita & Hawkins 

includeKeywords = ["Zavala", "Douglass", "Mesita", "Hawkins"]

dsa_str = '|'.join(includeKeywords)

mask = df.stack().str.contains(dsa_str).any(level=0)

df_masked = df[mask]


### --- Build hurly mean and median charts
source = df_masked

selection = alt.selection_multi(fields=['column_label'], bind='legend')

# set colorscale
domain = ['ry/Zavala 2 B (undefined) (31.771832 -106.447092) Primary ',
          'ry/Zavala E.S (outside) (31.771649 -106.447023) Primary ', 
          'ry/Zavala E.S B (undefined) (31.771649 -106.447023) Primary ',
          'ry/Zavala 2 (outside) (31.771832 -106.447092) Primary ',
          'ry/Mesita E.S (outside) (31.783542 -106.50389) Primary ',
          'ry/Mesita E.S B (undefined) (31.783542 -106.50389) Primary ',
          'ry/Mesita E.S (outside) (31.783542 -106.50389) Primary ',
          'ry/Hawkins E.S B (undefined) (31.777423 -106.419229) Primary ',
          'ry/Hawkins E.S (outside) (31.777423 -106.419229) Primary ',
          'ry/Douglass E.S. B (undefined) (31.766496 -106.464951) Primary ',
          'ry/Douglass E.S. (outside) (31.766496 -106.464951) Primary '
         ]

range_ = ['rgb(57, 59, 121)', 'rgb(82, 84, 163)', 'rgb(107, 110, 207)', 'rgb(156, 158, 222)', 
          'rgb(99, 121, 57)', 'rgb(140, 162, 82)',
          'rgb(132, 60, 57)', 'rgb(173, 73, 74)', 
          'rgb(140, 109, 49)', 'rgb(189, 158, 57)']

chart_mean_all = alt.Chart(source).mark_line().encode(
    alt.X('hour'),
    alt.Y('mean_pm2_5'),
    alt.Color('column_label', scale=alt.Scale(domain=domain, range=range_)),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.1))
).add_selection(
    selection
)


chart_median_all = alt.Chart(source).mark_line().encode(
    alt.X('hour'),
    alt.Y('median_pm2_5'),
    alt.Color('column_label', scale=alt.Scale(domain=domain, range=range_)),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.1))
).add_selection(
    selection
)


# Build faceted hourly charts

source_melt = pd.melt(df_masked, id_vars=['column_label', 'hour'], value_vars=['mean_pm2_5', 'median_pm2_5'])

facet_all = alt.Chart(source_melt).mark_line().encode(
    alt.X('hour', scale=alt.Scale(zero=False)),
    y='value',
    color='variable',
    facet=alt.Facet('column_label:O', columns=2),
).properties(
    width=200,
    height=100,
)


In [40]:
# Adjust filter to exclude Zavala 2 (outside) 

includeKeywords = ["Zavala E.S B", "Zavala E.S", "Zavala 2 B", "Douglass", "Mesita", "Hawkins"]

dsa_str = '|'.join(includeKeywords)

mask = df.stack().str.contains(dsa_str).any(level=0)

df_filtered = df[mask]



## Hourly mean and median charts without Zavala 2 (outside)
source = df_filtered

selection = alt.selection_multi(fields=['column_label'], bind='legend')

# set colorscale
domain = ['ry/Zavala 2 B (undefined) (31.771832 -106.447092) Primary ',
          'ry/Zavala E.S (outside) (31.771649 -106.447023) Primary ', 
          'ry/Zavala E.S B (undefined) (31.771649 -106.447023) Primary ',
          'ry/Mesita E.S (outside) (31.783542 -106.50389) Primary ',
          'ry/Mesita E.S B (undefined) (31.783542 -106.50389) Primary ',
          'ry/Mesita E.S (outside) (31.783542 -106.50389) Primary ',
          'ry/Hawkins E.S B (undefined) (31.777423 -106.419229) Primary ',
          'ry/Hawkins E.S (outside) (31.777423 -106.419229) Primary ',
          'ry/Douglass E.S. B (undefined) (31.766496 -106.464951) Primary ',
          'ry/Douglass E.S. (outside) (31.766496 -106.464951) Primary '
         ]

range_ = ['rgb(57, 59, 121)', 'rgb(82, 84, 163)', 'rgb(107, 110, 207)', 
          'rgb(99, 121, 57)', 'rgb(140, 162, 82)',
          'rgb(132, 60, 57)', 'rgb(173, 73, 74)', 
          'rgb(140, 109, 49)', 'rgb(189, 158, 57)']

chart_mean_filtered = alt.Chart(source).mark_line().encode(
    alt.X('hour'),
    alt.Y('mean_pm2_5'),
    alt.Color('column_label', scale=alt.Scale(domain=domain, range=range_)),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.1))
).add_selection(
    selection
)


chart_median_filtered = alt.Chart(source).mark_line().encode(
    alt.X('hour'),
    alt.Y('median_pm2_5'),
    alt.Color('column_label', scale=alt.Scale(domain=domain, range=range_)),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.1))
).add_selection(
    selection
)



# Build faceted chart

# Build faceted hourly median charts

source_melt_filtered = pd.melt(df_filtered, id_vars=['column_label', 'hour'], value_vars=['mean_pm2_5', 'median_pm2_5'])

facet_filtered = alt.Chart(source_melt_filtered).mark_line().encode(
    alt.X('hour', scale=alt.Scale(zero=False)),
    y='value',
    color='variable',
    facet=alt.Facet('column_label:O', columns=2),
).properties(
    width=200,
    height=100,
)

In [51]:
# Mean and median pm2.5 by hour for all Zavala, Hawkins, Douglass, and Mesita primary sensors
print('Mean and median pm2.5 by hour for all Zavala, Hawkins, Douglass, and Mesita monitors')

chart_mean_all | chart_median_all

Mean and median pm2.5 by hour for all Zavala, Hawkins, Douglass, and Mesita monitors


alt.HConcatChart(...)

In [50]:
# Mean and median pm2.5 by hour (excluding Zavala 2 (outside))
print('Mean and median pm2.5 by hour (excluding Zavala 2 (outside))')

chart_mean_filtered | chart_median_filtered

Mean and median pm2.5 by hour (excluding Zavala 2 (outside))


alt.HConcatChart(...)

In [49]:
# Mean and median pm2.5 by hour (excluding Zavala 2 (outside))
print('Mean and median pm2.5 by hour (excluding Zavala 2 (outside))')
facet_filtered

Mean and median pm2.5 by hour (excluding Zavala 2 (outside))


alt.Chart(...)